In [1]:
#!pip install sentence-transformers==2.2.2

In [2]:
#!pip install gpl

In [3]:
from vectorspace.store import Store as ESDict
from elasticsearch import Elasticsearch
corpus = ESDict(Elasticsearch(['http://localhost:9200']), 'bioasq')

In [4]:
import os
path_to_generated_data = os.path.expanduser("~/Downloads/bioasq")

In [6]:
# empty file because we have already indexed into ES
with open(f'{path_to_generated_data}/corpus.jsonl', 'w') as f:
    f.write('{}')

In [7]:
from beir.datasets.data_loader import GenericDataLoader
_, gen_queries, gen_qrels = GenericDataLoader(
    path_to_generated_data, prefix='qgen'
).load(split="train")

/opt/homebrew/anaconda3/lib/python3.9/site-packages/beir/datasets/data_loader.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
base_ckpt = 'michiyasunaga/BioLinkBERT-base'
pooling = None
max_seq_length = 512

In [9]:
from sentence_transformers import SentenceTransformer
from gpl.toolkit import load_sbert, GenerativePseudoLabelingDataset, MarginDistillationLoss
from torch.utils.data import DataLoader

2023-01-15 18:02:05 - Loading faiss.
2023-01-15 18:02:05 - Successfully loaded faiss.


In [11]:
#### It can load checkpoints in both SBERT-format (recommended) and Huggingface-format
model: SentenceTransformer = load_sbert(base_ckpt, pooling, max_seq_length)

gpl_training_data_fname = "gpl-training-data.tsv"
fpath_gpl_data = os.path.join(path_to_generated_data, gpl_training_data_fname)
train_dataset = GenerativePseudoLabelingDataset(
    fpath_gpl_data, gen_queries, corpus
)
train_dataloader = DataLoader(
    train_dataset, shuffle=False, batch_size=32, drop_last=True
)  # Here shuffle=False, since (or assuming) we have done it in the pseudo labeling

[2023-01-15 18:02:08] INFO [sentence_transformers.SentenceTransformer.__init__:66] Load pretrained SentenceTransformer: michiyasunaga/BioLinkBERT-base
[2023-01-15 18:02:13] WARNING [sentence_transformers.SentenceTransformer._load_auto_model:805] No sentence-transformers model found with name /Users/mr/.cache/torch/sentence_transformers/michiyasunaga_BioLinkBERT-base. Creating a new one with MEAN pooling.
[2023-01-15 18:02:14] INFO [sentence_transformers.SentenceTransformer.__init__:105] Use pytorch device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[2023-01-15 18:02:14] INFO [gpl.toolkit.sbert.load_sbert:68] Set max_seq_length=512


In [12]:
output_dir = path_to_generated_data

In [13]:
gpl_score_function = "dot"
train_loss = MarginDistillationLoss(
    model=model, similarity_fct=gpl_score_function
)

gpl_steps = 140000
# assert gpl_steps > 1000
model.fit(
    [
        (train_dataloader, train_loss),
    ],
    epochs=1,
    steps_per_epoch=gpl_steps,
    warmup_steps=1000,
    checkpoint_save_steps=10000,
    checkpoint_save_total_limit=10000,
    output_path=output_dir,
    checkpoint_path=output_dir,
    #use_amp=use_amp,
)

[2023-01-15 18:02:14] INFO [gpl.toolkit.loss.__init__:24] Set GPL score function to dot


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/140000 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/sc/18nvh9y94q9fy8hzdrzkl8lh0000gn/T/ipykernel_88024/3552277691.py:8 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/sc/18nvh9y94q9fy8hzdrzkl8lh0000gn/T/ipykernel_88024/3552277691.py'                 │
│                                                                                                  │
│ /opt/homebrew/anaconda3/lib/python3.9/site-packages/sentence_transformers/SentenceTransformer.py │
│ :721 in fit                                                                                      │
│                                                                                                  │
│   718 │   │   │   │   │   │                                                                      │
│   719 │   │   │   │   │   │   skip_scheduler = scaler.get_scale() != scale_before_step           │
│   720 │   │   │   │   │   else:                                                                  │
│ ❱ 721 │   │   │   │   │   │   loss_value = loss_model(features, labels)                          │
│   722 │   │   │   │   │   │   loss_value.backward()                                              │
│   723 │   │   │   │   │   │   torch.nn.utils.clip_grad_norm_(loss_model.parameters(), max_grad   │
│   724 │   │   │   │   │   │   optimizer.step()                                                   │
│                                                                                                  │
│ /opt/homebrew/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py:1482 in           │
│ _call_impl                                                                                       │
│                                                                                                  │
│   1479 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1480 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1481 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1482 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1483 │   │   # Do not call functions when jit is used                                          │
│   1484 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1485 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /opt/homebrew/anaconda3/lib/python3.9/site-packages/gpl/toolkit/loss.py:29 in forward            │
│                                                                                                  │
│   26 │                                                                                           │
│   27 │   def forward(self, sentence_features: Iterable[Dict[str, Tensor]], labels: Tensor):      │
│   28 │   │   # sentence_features: query, positive passage, negative passage                      │
│ ❱ 29 │   │   reps = [                                                                            │
│   30 │   │   │   self.model(sentence_feature)["sentence_embedding"]                              │
│   31 │   │   │   for sentence_feature in sentence_features                                       │
│   32 │   │   ]                                                                                   │
│                                                                                                  │
│ /opt/homebrew/anaconda3/lib/python3.9/site-packages/gpl/toolkit/loss.py:30 in <listcomp>         │
│                                                            